In [49]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplot.pyplot as plt

In [2]:
file_k = "Data/kunden.csv"
file_b = "Data/besucher.csv"
file_geo = "Data/geo.txt"

In [3]:
df_k = pd.read_csv(file_k, sep = ";", decimal = ".")
df_b = pd.read_csv(file_b, sep = ";", decimal = ",")
df_geo = pd.read_table(file_geo, delimiter = "\t") #\t muss verwendet werden damit die Spalten der Liste aus der txt Datei separiert werden können

In [4]:
df_geo

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern
...,...,...
1990,K1991,Hessen
1991,K1992,Baden-Württemberg
1992,K1993,Nordrhein-Westfalen
1993,K1994,Berlin


In [5]:
df_b_k = pd.concat([df_b,df_k], ignore_index = True)
df_b_k

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis
0,30,39919.095335,0.0,29.0,K1814,NaN
1,41,53430.673086,0.0,27.0,K1544,NaN
2,64,73163.216927,0.0,32.0,K1945,NaN
3,41,36761.482581,0.0,44.0,K1825,NaN
4,35,42684.331632,1.0,33.0,K1144,NaN
...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58
1991,38,48301.000000,1.0,33.0,K0122,64230.98
1992,67,63547.000000,1.0,41.0,K1045,103123.57
1993,37,32686.000000,1.0,36.0,K1096,49215.87


In [6]:
df_bkg = pd.merge(df_b_k, df_geo, on = "KundeNr")
df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen
...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg


In [7]:
# Berechnung von Q1 (25. Perzentil), Q3 (75. Perzentil) und IQR
Q1 = df_bkg['Einkommen'].quantile(0.25)
Q3 = df_bkg['Einkommen'].quantile(0.75)
IQR = Q3 - Q1

# Berechnung der Ausreißer-Grenzen
untere_grenze = Q1 - 1.6 * IQR #Faktor auf die Ausreißer nach eigenem Ermessen angepasst
obere_grenze = Q3 + 5 * IQR #Faktor auf die Ausreißer nach eigenem Ermessen angepasst

# Identifizierung von Ausreißern
df_bkg['Ausreißer'] = (df_bkg['Einkommen'] < untere_grenze) | (df_bkg['Einkommen'] > obere_grenze)

df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern,False
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg,False
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg,False
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg,False
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen,False
...,...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen,False
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen,False
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin,False
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg,False


In [8]:
untere_grenze

9204.719763225141

In [9]:
#Median des Einkommens bestimmen 
einkommen_medianwert = df_bkg["Einkommen"].median()
einkommen_medianwert 

59104.51301372704

In [10]:
#Überprüfung des Median
Q2 = df_bkg['Einkommen'].quantile(0.5)
Q2

59104.51301372704

In [11]:
#Ersetzen der oberen und unteren Werte
df_bkg.loc[df_bkg["Einkommen"] < untere_grenze, "Einkommen"] = einkommen_medianwert
df_bkg.loc[df_bkg["Einkommen"] > obere_grenze, "Einkommen"] = einkommen_medianwert

In [12]:
df_bkg.iloc[1299]

Alter                             68
Einkommen               59104.513014
Geschlecht                       0.0
Zeit                            52.0
KundeNr                        K0701
Preis                       87209.72
Niederlassung    Nordrhein-Westfalen
Ausreißer                       True
Name: 1299, dtype: object

In [13]:
df_bkg.iloc[1857, 1]

59104.51301372704

In [14]:
#Identifikation fehlender Werte bei Kategorie Geschlecht
geschlecht_NAN = df_bkg[df_bkg["Geschlecht"].isna()]
geschlecht_NAN

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
1001,41,46542.0,NaN,40.0,K0024,63477.00,Baden-Württemberg,False
1203,53,49798.0,NaN,39.0,K0166,61329.51,Nordrhein-Westfalen,False
1479,64,83452.0,NaN,62.0,K0090,133180.70,Niedersachsen,False
1503,52,66602.0,NaN,45.0,K0325,98756.07,Bayern,False
1622,67,91461.0,NaN,66.0,K0790,131748.07,Berlin,False


In [15]:
#Identifikation des Modalwert bei der Kategorie Geschlecht
geschlecht_modalwert = df_bkg["Geschlecht"].mode()[0]
geschlecht_modalwert 

1.0

In [16]:
#Geschlechtswerte ohne Eintrag durch Modalwert ersetzen
df_bkg["Geschlecht"] = df_bkg["Geschlecht"].fillna(geschlecht_modalwert)
df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern,False
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg,False
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg,False
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg,False
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen,False
...,...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen,False
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen,False
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin,False
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg,False


In [17]:
#Identifikation von wahrscheinlich falschen oder gar illegalen Altersangaebn
unglaubwürdiges_alter = df_bkg[(df_bkg["Alter"] > 90) | (df_bkg["Alter"] < 18)]
unglaubwürdiges_alter

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
1033,100,88749.0,0.0,64.0,K0494,119134.23,Bayern,False
1602,120,61857.0,1.0,52.0,K0039,104214.07,Baden-Württemberg,False


In [18]:
#Identifikation des Modalwert bei der Kategorie Alter
alter_medianwert = df_bkg["Alter"].median()
alter_medianwert 

51.0

In [19]:
#Bereinigung Alter und Überprüfung
df_bkg.loc[df_bkg["Alter"] > 90, "Alter"] = alter_medianwert
df_bkg.iloc[1602]

Alter                           51
Einkommen                  61857.0
Geschlecht                     1.0
Zeit                          52.0
KundeNr                      K0039
Preis                    104214.07
Niederlassung    Baden-Württemberg
Ausreißer                    False
Name: 1602, dtype: object

In [20]:
#Definition der Bundesländer
bundesländer = ["Baden-Württemberg", "Bayern", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hessen",
                "Mecklenburg-Vorpommern", "Niedersachsen", "Nordrhein-Westfalen", "Rheinland-Pfalz", "Saarland",
                "Sachsen", "Sachsen-Anhalt", "Schleswig-Holstein", "Thüringen"]
#Nur jene Einträge für Niederlassungen anzeigen lassen, welche *nicht* in vorheriger Liste vorhanden sind
bundesländer_falsche_namen = df_bkg[~df_bkg["Niederlassung"].isin(bundesländer)]
bundesländer_falsche_namen

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
299,44,54025.601631,1.0,30.0,K1987,NaN,NRW,False
491,49,54754.525450,1.0,44.0,K1995,NaN,Berlin-Charlottenburg,False
587,63,84772.480847,1.0,30.0,K1973,NaN,Düsseldorf,False
840,32,40776.023145,1.0,13.0,K1609,NaN,BERLIN,False
911,43,52813.000000,0.0,44.0,K0762,67106.32,BERLIN,False
1249,61,56063.000000,1.0,44.0,K0679,98148.54,Berlin-Mitte,False
1472,55,66738.000000,1.0,52.0,K0413,120215.50,Düsseldorf,False
1510,62,75575.000000,1.0,50.0,K0023,124516.56,NRW,False
1783,58,65271.000000,1.0,44.0,K0002,106258.99,NRW,False
1812,30,28784.000000,1.0,34.0,K0677,42416.07,Berlin-Mitte,False


In [21]:
#Jetzt ersetzen wir die ungewollten Bezeichnungen der Niederlassungen mit den gewollten Bezeichnungen
#1. den "falschen" Schlüsseln die gewollten Werte zuordnen
ersetzen_dict = {
    "Berlin-Charlottenburg": "Berlin",
    "BERLIN": "Berlin",
    "Berlin-Mitte": "Berlin",
    "Düsseldorf": "Nordrhein-Westfalen",
    "NRW": "Nordrhein-Westfalen",
}
#2. Die geändert Werte jetzt in das Datafram übertragen
df_bkg["Niederlassung"].replace(ersetzen_dict, inplace=True)

In [22]:
#Überprüfung ob korrekt ersetzt wurde (Stichprobe)
df_bkg.iloc[1472, 6]

'Nordrhein-Westfalen'

In [23]:
df_bkg.to_csv("df_final.csv", index=False)

In [24]:
#F1 Wie viele Autos wurden verkauft?
df_preis_vorhanden = df_bkg[df_bkg["Preis"].notna()]
df_autos_verkauft = df_preis_vorhanden["KundeNr"].nunique()
df_autos_verkauft

1104

In [25]:
#F3 Wie hoch ist der Gesamtumsatz?
df_umsatz = df_bkg["Preis"].dropna().sum()
df_umsatz

98228619.38000001

In [26]:
#F5 Wie hoch war der durchschnittliche Umsatz pro Bundesland?
#Ermittelung der Spalten wo Preis (siehe F1)
#Ermittlung Durchschnittsumsatz pro Bundesland
durchschnitt_umsatz_pro_land = df_preis_vorhanden.groupby('Niederlassung')['Preis'].mean()
durchschnitt_umsatz_pro_land

Niederlassung
Baden-Württemberg      90969.560000
Bayern                 89344.313245
Berlin                 89584.984048
Brandenburg            86673.640811
Hamburg                85549.499070
Hessen                 85864.929479
Niedersachsen          87963.312500
Nordrhein-Westfalen    88633.265200
Sachsen                89455.080674
Thüringen              90816.994082
Name: Preis, dtype: float64

In [32]:
#F7 Haben mehr Frauen oder mehr Männer unsere Autos gekauft?
df_preis_vorhanden["Geschlecht"].mode()

0    1.0
Name: Geschlecht, dtype: float64

In [28]:
#F9 Wie hoch ist das Durchschnittseinkommen unserer Kunden? 
df_preis_vorhanden["Einkommen"].mean()

58895.24686507354

In [29]:
#F9 Wie hoch ist das Durchschnittseinkommen unserer Besucher?
#Ermittlungen der Zeilen mit NAN in der Preis-Spalte
df_preis_nicht_vorhanden = df_bkg[df_bkg["Preis"].isna()]
#Durchschnittseinkommen der Besucher berechenen
df_preis_nicht_vorhanden["Einkommen"].mean()

59367.93618779704

In [30]:
#F11 Wie hoch ist die Korrelation (Person-Korrelation) zwischen den Variablen Alter, Einkommen, Preis und Zeit?
corr_kundenwerte = df_preis_vorhanden[["Alter", "Einkommen", "Preis", "Zeit"]].corr()
corr_kundenwerte

,Alter,Einkommen,Preis,Zeit
Alter,1.000000,0.761572,0.780478,0.588912
Einkommen,0.761572,1.000000,0.896565,0.776211
Preis,0.780478,0.896565,1.000000,0.707778
Zeit,0.588912,0.776211,0.707778,1.000000


In [47]:
corr_kundenwerte2 = df_preis_vorhanden[["Alter", "Einkommen", "Preis", "Zeit", "Geschlecht",]].corr()
corr_kundenwerte2

,Alter,Einkommen,Preis,Zeit,Geschlecht
Alter,1.000000,0.761572,0.780478,0.588912,0.009515
Einkommen,0.761572,1.000000,0.896565,0.776211,0.033953
Preis,0.780478,0.896565,1.000000,0.707778,0.068424
Zeit,0.588912,0.776211,0.707778,1.000000,0.021506
Geschlecht,0.009515,0.033953,0.068424,0.021506,1.000000


In [42]:
#F13 Wie viele Kunden haben keinen Bankkredit aufgenommen, um das Auto zu kaufen?
#Filterung nach Einkommen-Spalte, wobei in Zeile: Einkommen > Preis
df_kein_kredit = df_preis_vorhanden.loc[df_preis_vorhanden["Einkommen"] > df_preis_vorhanden["Preis"], "Einkommen"]
df_kein_kredit

1048    50330.0
1298    24264.0
1390    54403.0
1533    30856.0
1587    36296.0
Name: Einkommen, dtype: float64

In [43]:
#F13 Wie viele Kunden haben keinen Bankkredit aufgenommen, um das Auto zu kaufen?
len(df_kein_kredit)

5

In [ ]:
#F14 Welche sozioökonomischen Merkmale beeinflussen den Kaufpreis?
#Durch die Korrelationswerte aus Aufgabe 11 liegt es nahe, dass Einkommen, Alter positiv mit dem Kaufpreis korrelieren

In [65]:
#Neuen Dataframe mit relevanten unabhängigen Variablen erstellen
nicht_rel_var = ["KundeNr", "Niederlassung", "Ausreißer", "Preis"]
df_rel_var = df_preis_vorhanden.drop(columns = nicht_rel_var)
df_rel_var.head()

,Alter,Einkommen,Geschlecht,Zeit
891,64,66894.0,1.0,43.0
892,54,77644.0,1.0,40.0
893,55,44341.0,0.0,37.0
894,49,67271.0,0.0,42.0
895,46,49832.0,0.0,41.0


In [67]:
y = df_preis_vorhanden["Preis"]
X = df_rel_var
X = sm.add_constant(X)

In [70]:
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Preis   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     1329.
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:07:21   Log-Likelihood:                -11748.
No. Observations:                1104   AIC:                         2.351e+04
Df Residuals:                    1099   BIC:                         2.353e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -128.4750   1532.376     -0.084      0.933   -3135.188    2878.238
Alter        479.7397     39.447     12.162      0.000     402.340     557.140
Einkommen      1.0256      0.037     28.005      0.000       0.954       1.097
Geschlecht  2139.7464    636.427      3.362      0.001     890.997    3388.496
Zeit          71.3735     44.660      1.598      0.110     -16.255     159.002
==============================================================================
Omnibus:                        1.250   Durbin-Watson:                   2.103
Prob(Omnibus):                  0.535   Jarque-Bera (JB):                1.252
Skew:                           0.014   Prob(JB):                        0.535
Kurtosis:                       2.837   Cond. No.                     3.09e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#Schlussfolgerungen:
Da der P>|t|-Wert bei der Zeit den Wert 0.05 überschreitet, ist die Zeit ein nicht signifikanter Koeffizient. Daher kann nicht gesagt werden, ob die Zeit einen entscheidenden Einfluss auf den Kaufpreis hat.
Dahingegen sind Einkommen, Alter und Geschlecht signifikant.

In [89]:
X

,const,Alter,Einkommen,Geschlecht,Zeit
891,1.0,64,66894.0,1.0,43.0
892,1.0,54,77644.0,1.0,40.0
893,1.0,55,44341.0,0.0,37.0
894,1.0,49,67271.0,0.0,42.0
895,1.0,46,49832.0,0.0,41.0
...,...,...,...,...,...
1990,1.0,70,69581.0,1.0,51.0
1991,1.0,38,48301.0,1.0,33.0
1992,1.0,67,63547.0,1.0,41.0
1993,1.0,37,32686.0,1.0,36.0


In [105]:
#F15 Prognose - männlicher Kunden im Alter von 32 Jahren mit einem Einkommen von 30.000 Euro
y1 = pd.DataFrame({"Alter" : [32], "Einkommen" : [30000], "Geschlecht" : [1], "Zeit" : [df_preis_vorhanden["Zeit"].mean()]})

In [106]:
#y1 = sm.add_constant(y1) funktioniert nicht, weshalb die predict Funktion nicht funktioniert. Ich füge die Konstante manuell hinzu
y1.insert(0, "const", 1)

In [107]:
y1

,const,Alter,Einkommen,Geschlecht,Zeit
0,1,32,30000,1,44.375906


In [109]:
Prognose1 = model.predict(y1)
Prognose1

0    51297.291522
dtype: float64

In [110]:
#F15 Prognose - männlicher Kunden im Alter von 51 Jahren mit einem Einkommen von 54.000 Euro
y2 = pd.DataFrame({"Alter" : [51], "Einkommen" : [54000], "Geschlecht" : [1], "Zeit" : [df_preis_vorhanden["Zeit"].mean()]})
y2.insert(0, "const", 1)
Prognose2 = model.predict(y2)
Prognose2

0    85026.014079
dtype: float64

#F19 Auf welche Probleme sind Sie bei der Zusammenführung des Datensatzes gestoßen?
0. Wir waren uns zu Beginn unsicher, ob wir die Listen direkt in dem Dataframe zusammenfügen sollen und damit weiterarbeiten (so haben wir es letztendlich gemacht) oder erstmal die Listen der Besucher und Kunden zusammenführen (mit pandas), daraus eine Excel-Datei machen und diese Excelliste dann mit der geo.txt Datei verbinden und auch daraus eine "finale" Exel-Datei zu machen, auf welche wir zugreifen würden. Die 2 Option war der anfängliche Ansatz, aber schien zu umständlich. 
1. Es war herausfordernd herauszufinden, welche der Funktionen (concat oder merger) zuerst angewendet werden soll bzw. welcher sich besser eignet. 
2. Ein weiteres Problem war zu verstehen, welche inputs die Funktion Concat benötigt, um die Listen so zusammenzufügen wie man es will. Die Lösung ist im Endeffekt sehr simpel, aber ohne vorheriges Verständnis von der funktionsweise der Funktion, dauert es etwas die Lösung nachzuvollziehen.
3. Ein ähnliches Problem gab es auch bei der merge-Funktion. Es gibt viele mögliche Variablen die bestimmt werden können und bis man herausgefunden hat, welche man benutzen muss, um die Excelliste und die Weise wie die Liste am Ende aussehen soll, zu erreichen, dauert das eine Weile.